In [ ]:
rank = 500

def dummy_old_grad(obj_idx, new_grad):
    old_grad = dummy_old_grad.g[obj_idx, :].copy()
    dummy_old_grad.g[obj_idx, :] = new_grad.copy()
    u, s, v = np.linalg.svd(g)
    u = u[:, :rank]
    s = s[:rank]
    v = v[:rank, :]
    dummy_old_grad.g = u.dot(np.diag(s)).dot(v)
    return old_grad
dummy_old_grad.g = np.zeros(n_objects, n_features)


svd_size = 5000
def folding_in_old_grad(obj_idx, new_grad):
    if folding_in_old_grad.svd_is_ready:
        old_grad = folding_in_old_grad.u[obj_idx, :].dot(folding_in_old_grad.v)
        folding_in_old_grad.u[obj_idx, :] = new_grad.dot(folding_in_old_grad.v.T)
    else:
        if obj_idx in folding_in_old_grad.objects_seen:
            g_part_idx = folding_in_old_grad.objects_seen.index(obj_idx)
        else:
            g_part_idx = folding_in_old_grad.n_objects_seen
            folding_in_old_grad.objects_seen[g_part_idx] = obj_idx
            folding_in_old_grad.n_objects_seen += 1
        old_grad = folding_in_old_grad.g_part[g_part_idx, :].copy()
        folding_in_old_grad.g_part[g_part_idx, :] = new_grad.copy()
        if folding_in_old_grad.n_objects_seen >= svd_size:
            u, s, v = np.linalg.svd(folding_in_old_grad.g_part)
            u = u[:, :rank]
            s = s[:rank]
            v = v[:rank, :]
            folding_in_old_grad.u = np.zeros(n_objects, rank)
            folding_in_old_grad.u[folding_in_old_grad.objects_seen, :] = u.dot(np.diag(s))
            folding_in_old_grad.v = v
            folding_in_old_grad.svd_is_ready = True
    return old_grad
folding_in_old_grad.objects_seen = -1 * np.ones(svd_size)
folding_in_old_grad.n_objects_seen = 0
folding_in_old_grad.g_part = np.zeros(svd_size, rank)
folding_in_old_grad.svd_is_ready = False


    